### Libraries

In [1]:
import os
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import re
import collections
import string
import math
%matplotlib inline

### Read data

In [25]:
filename = './data/prepr_train_character.csv'
train_data_ = pd.read_csv(filename)
print ('Data size:', len(train_data_))
train_data_.head()

Data size: 3947


,Insult,Date,Comment
0,1,20120618192155Z,you fuck your dad
1,0,20120528192215Z,i really do not understand your point xa it se...
2,0,NaN,a xc xa majority of canadians can and has been...
3,0,NaN,listen if you dont wanna get married to a man ...
4,0,20120619094753Z,c xe c b u ea n xu u ed ng u u b u eddng bi u ...


In [26]:
filename = './data/prepr_valid_character.csv'
valid_data_ = pd.read_csv(filename)
print ('Data size:', len(valid_data_))
valid_data_.head()

Data size: 2647


,Insult,Date,Comment,Usage
0,0,NaN,the drudge report n n n nyou wo not see this s...,PublicTest
1,0,20120618222256Z,ian xa roger clemens is the fucking man and ne...,PublicTest
2,1,20120618213617Z,agree with alan you are an extremest idiot you...,PublicTest
3,0,NaN,really n ni see marc lamont hill on various pr...,PrivateTest
4,0,20120620003825Z,really suck is not the word when many of our n...,PrivateTest


In [27]:
filename = './data/prepr_test_character.csv'
test_data_ = pd.read_csv(filename)
print ('Data size:', len(test_data_))
test_data_.head()

Data size: 2235


,Insult,Date,Comment
0,0,20120603163526Z,like this if you are a tribe fan
1,1,20120531215447Z,you idiot
2,1,20120823164228Z,i am a woman babs and the only war on women i ...
3,1,20120826010752Z,wow you benefitted so many wins this year from...
4,1,20120602223825Z,haha green me red you now loser whos winning n...


In [28]:
train_data = train_data_
valid_data = valid_data_
test_data = test_data_
                               
valid_size = len(valid_data)
train_size = len(train_data)
test_size = len(test_data)

X_train = train_data['Comment']
y_train = train_data['Insult']
X_valid = valid_data['Comment']
y_valid = valid_data['Insult']
X_test = test_data['Comment']
y_test = test_data['Insult']

In [29]:
vocabulary_size = len(string.ascii_lowercase) + 1 + 1 + 1  # [a-z] + ' ' + END + PAD
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ':
        return 0
    elif char == 'E':
        return 27
    elif char == 'P':
        return 28
    else:
        print('Unexpected character: %s' % char)
        return 0
    
def id2char(dictid):
    if dictid > 0 and dictid < 27: # id(z) = 26
        return chr(dictid + first_letter - 1)
    elif dictid == 27:
        return 'E'
    elif dictid == 28:
        return 'P'
    else:
        return ' '

def comment2vec(comment_, comment_size):
    N = len(comment_)
    if N > comment_size - 2:
        comment = comment_[:comment_size - 1] + 'E'
    else:
        comment = comment_ + 'E' + 'P' * (comment_size - N - 1)
    res = np.array([char2id(char) for char in comment], dtype='int')
    return res
    
print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


### Word indexing

In [30]:
class LabeledBatchGenerator(object):
    def __init__(self, comments, batch_size, comment_size, labels):
        self._comments = comments
        self._num_comments = len(comments)
        self._batch_size = batch_size
        self._comment_size = comment_size
        self._labels = labels
        segment = self._num_comments // batch_size
        self._cursor = [offset * segment for offset in range(batch_size)]
        
    def _next_batch(self, step):
        batch = np.zeros(shape=(self._batch_size,1), dtype=np.int)
        for b in range(self._batch_size):
            comment = comment2vec(self._comments[self._cursor[b]], self._comment_size)
            batch[b,0] = comment[step]
            if step == self._comment_size - 1:
                self._cursor[b] = (self._cursor[b] + 1) % self._num_comments
        return batch
    
    def next(self):
        batches = []
        batches_labels = [self._labels[self._cursor[b]] for b in range(self._batch_size)]
        for step in range(0, self._comment_size):
            batches.append(self._next_batch(step))
        return batches, batches_labels

In [31]:
comment_size = 100
batch_size = 64

train_batches, train_labels = LabeledBatchGenerator(X_train.as_matrix(), train_size, comment_size, y_train.as_matrix()).next()
valid_batches, valid_labels = LabeledBatchGenerator(X_valid.as_matrix(), valid_size, comment_size, y_valid.as_matrix()).next()
test_batches, test_labels = LabeledBatchGenerator(X_test.as_matrix(), test_size, comment_size, y_test.as_matrix()).next()

In [ ]:
X_train = np.asarray(train_batches).reshape(comment_size, train_size).T
y_train = np.asarray(train_labels).reshape(-1)
X_valid = np.asarray(valid_batches).reshape(comment_size, valid_size).T
y_valid = np.asarray(valid_labels).reshape(-1)
X_test = np.asarray(test_batches).reshape(comment_size, test_size).T
y_test = np.asarray(test_labels).reshape(-1)

In [ ]:
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Input, concatenate, Activation, Bidirectional
from keras.layers.embeddings import Embedding
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.optimizers import Adam, SGD
from keras.layers.core import Dropout, Reshape
from keras.regularizers import l2

In [ ]:
embedding_size = 128
num_nodes = 32

inputs = Input(shape=(comment_size,))
embed_inputs = Embedding(vocabulary_size, embedding_size,
                         trainable=True)(inputs)

lstm_0 = Bidirectional(LSTM(num_nodes, kernel_regularizer=l2(0.01), dropout=0.1))(embed_inputs)
print ('Bidirectional LSTM:', lstm_0.shape)
#lstm_1 = Bidirectional(LSTM(num_nodes, kernel_regularizer=l2(0.01), dropout=0.1))(embed_inputs)

#lstm_2 = Bidirectional(LSTM(num_nodes, kernel_regularizer=l2(0.01), dropout=0.1))(embed_inputs)

#concat_layers = concatenate([lstm_0, lstm_1, lstm_2], axis=1)
concat_layers = lstm_0
print('Concatenated:', concat_layers.shape)

concat_layers = Dropout(rate=0.1)(concat_layers)
output = Dense(1, activation=None, kernel_regularizer=l2(0.01))(concat_layers) 
act_output = Activation('sigmoid')(output)
print ('Final:',act_output.shape)
model = Model(inputs=inputs, outputs=act_output)

optimizer = Adam(lr=0.01, decay=0.1)

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

print (model.summary())
print (model.output_shape)

print('Train...')
model.fit(X_train, y_train.reshape(-1,1),
          batch_size=batch_size,
          epochs=10,
          validation_data=(X_valid, y_valid.reshape(-1,1)))
score, acc = model.evaluate(X_valid, y_valid.reshape(-1,1),
                            batch_size=batch_size)

model.save_weights(filepath='./checkpoint_dir/wordLSTMkeras')

print('Valid score:', score)
print('Valid accuracy:', acc)